<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/chemistry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Quantum (Computational) Chemistry**

In [ ]:
import sympy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Quantum Computing Libraries
!pip install cirq --quiet
import cirq
from cirq.contrib.svg import SVGCircuit
print(cirq.__version__)

### **<font color="blue">Quantum Chemistry $\hat{H}_{elec}$**

https://pennylane.ai/qml/demos/tutorial_quantum_chemistry.html

###### **Mean-field theory**

* In physics and probability theory, [mean-field theory](https://en.m.wikipedia.org/wiki/Mean-field_theory) (aka MFT or rarely self-consistent field theory) studies the behavior of high-dimensional random (stochastic) models by studying a simpler model that approximates the original by averaging over degrees of freedom (the number of values in the final calculation of a statistic that are free to vary). Such models consider many individual components that interact with each other. 

* In MFT, the effect of all the other individuals on any given individual is approximated by a single averaged effect, **thus reducing a [many-body problem](https://en.m.wikipedia.org/wiki/Many-body_problem) to a [one-body problem (two body)](https://en.m.wikipedia.org/wiki/Two-body_problem)**.

###### **Atomic orbitals & Molecular orbital**

* a [molecular orbital](https://en.m.wikipedia.org/wiki/Molecular_orbital) is a mathematical function **describing the location and wave-like behavior of an electron in a molecule**. 

* This function can be used to calculate chemical and physical properties such as the probability of finding an electron in any specific region. 

* The terms atomic orbital and molecular orbital were introduced by Robert S. Mulliken in 1932 to mean one-electron orbital wave functions. At an elementary level, they are **used to describe the region of space in which a function has a significant amplitude**.

* In an isolated atom, the orbital electrons' location is determined by functions called [atomic orbitals](https://en.m.wikipedia.org/wiki/Atomic_orbital). When multiple atoms combine chemically into a molecule, the electrons' locations are determined by the molecule as a whole, so **the atomic orbitals combine to form molecular orbitals**. The electrons from the constituent atoms occupy the molecular orbitals. 

* Mathematically, **molecular orbitals are an approximate solution to the Schrödinger equation for the electrons in the field of the molecule's [atomic nuclei](https://en.m.wikipedia.org/wiki/Atomic_nucleus)**. 

* They are usually constructed by combining [atomic orbitals](https://en.m.wikipedia.org/wiki/Atomic_orbital) or [hybrid orbitals](https://en.m.wikipedia.org/wiki/Orbital_hybridisation) from each atom of the molecule, or other molecular orbitals from groups of atoms. **They can be quantitatively calculated using the Hartree–Fock or self-consistent field (SCF) methods**.




Molecular orbitals are of three types: 

* bonding orbitals which have an energy lower than the energy of the atomic orbitals which formed them, and thus promote the chemical bonds which hold the molecule together; 

* antibonding orbitals which have an energy higher than the energy of their constituent atomic orbitals, and so oppose the bonding of the molecule, and 

* non-bonding orbitals which have the same energy as their constituent atomic orbitals and thus have no effect on the bonding of the molecule.




A molecular orbital (MO) can be used to **represent the regions in a molecule where an electron occupying that orbital is likely to be found**. 

* Molecular orbitals are approximate solutions to the Schrödinger equation for the electrons in the electric field of the molecule's atomic nuclei. 

* However calculating the orbitals directly from this equation is far too intractable a problem. Instead **they are obtained from the combination of atomic orbitals, which predict the location of an electron in an atom**. 

* A molecular orbital can specify the electron configuration of a molecule: 

  * the spatial distribution and energy of one (or one pair of) electron(s). 
  
  * Most commonly a MO is represented as a linear combination of atomic orbitals (the LCAO-MO method), especially in qualitative or very approximate usage. They are invaluable in providing a simple model of bonding in molecules, understood through molecular orbital theory. Most present-day methods in computational chemistry begin by calculating the MOs of the system. 
  
  * **A molecular orbital describes the behavior of one electron in the electric field generated by the nuclei and some average distribution of the other electrons**. In the case of two electrons occupying the same orbital, the Pauli principle demands that they have opposite spin. Necessarily this is an approximation, and highly accurate descriptions of the molecular electronic wave function do not have orbitals (see configuration interaction).



See also example: https://en.wikipedia.org/wiki/Trigonal_planar_molecular_geometry

###### **Molekülorbitaltheorie (MO-Theorie)**

* Die [Molekülorbitaltheorie](https://de.m.wikipedia.org/wiki/Molek%C3%BClorbitaltheorie) (kurz MO-Theorie) bzw. [Molecular orbital theory](https://en.m.wikipedia.org/wiki/Molecular_orbital_theory) ist neben der Valenzbindungstheorie (VB-Theorie) eine von zwei komplementären Möglichkeiten, <font color="blue">**die Elektronenstruktur von Molekülen zu beschreiben**.</font>

* Beim MO-Verfahren werden über das Molekül delokalisierte Molekülorbitale durch eine Linearkombination ([LCAO-Methode](https://de.m.wikipedia.org/wiki/LCAO-Methode)) der Atomorbitale aller Atome eines Moleküls gebildet. 

* Dabei wird zwischen bindenden und antibindenden Molekülorbitalen unterschieden. 

* Das Verfahren wird heute für die meisten quantenchemischen Rechnungen verwendet.


Eine zeitunabhängige $n$-Elektronen-Wellenfunktion hat, wenn der Spin nicht beachtet wird, die allgemeine Form $\Psi\left(\vec{r}_{1}, \ldots \vec{r}_{n}\right)$. 

* Das Produkt mit der komplex konjugierten Funktion $\Psi^{*}\left(\vec{r}_{1}, \ldots \vec{r}_{n}\right) \cdot \Psi\left(\vec{r}_{1}, \ldots \vec{r}_{n}\right)$ ist reell und gibt die Wahrscheinlichkeitsdichte wieder, das erste Elektron an der Stelle $\vec{r}_{1}$, das 2-te an der Stelle $\vec{r}_{2}$ usw. zu finden.


* <font color="blue">**Die exakte Wellenfunktion lässt sich für Mehrelektronensysteme analytisch nicht finden. Eine zielführende Vereinfachung ist es, die Elektronen als stochastisch unabhängig anzusehen**</font>. Mathematisch bedeutet das, einen Produktansatz zu verwenden $\Psi\left(\vec{r}_{1}, \ldots \vec{r}_{n}\right)=\prod_{i=1}^{n} \Psi_{i}\left(\vec{r}_{i}\right)$. Dieser Ansatz ist auch als Hartree-Produkt bekannt. <font color="blue">**Die $\Psi_{i}\left(\vec{r}_{i}\right)$ geben die Aufenthaltsbereiche für die einzelnen Elektronen an**</font>. Sie werden als Molekülorbitale bezeichnet. 

* Um das Pauli-Prinzip einzuhalten, wird die Wellenfunktion als [Slater-Determinante](https://de.m.wikipedia.org/wiki/Slater-Determinante) (ein antisymmetrisierter Produktansatz) angesetzt. Dabei ist gewährleistet, dass die Wellenfunktion [antisymmetrisch](https://de.m.wikipedia.org/wiki/Antisymmetrische_Funktion) unter Vertauschung zweier ununterscheidbarer Elektronen ist, wie es für Fermionen gelten muss.

Abgesehen davon, **dass MO-Schemata eine vereinfachte, modellhafte Beschreibung darstellen**, ist zu beachten, dass sie in der MO-Theorie nicht eindeutig bestimmt sind. 

* Entscheidend ist nur die Summe über alle quadrierten Orbitale, die Elektronendichte (das ist auch die Grundlage für die [Dichtefunktionaltheorie DFT](https://de.m.wikipedia.org/wiki/Dichtefunktionaltheorie_(Quantenphysik))). Mathematisch gesprochen ist die Wellenfunktion gegenüber einer unitären Lineartransformation invariant. 

* So lassen sich neben den oft als kanonische MOs bezeichneten Orbitale verschieden andere Darstellungen finden, z. B. Localized Molecular Orbitals (LMOs).

###### **Valenzbindungstheorie (Valence bond theory)**

https://de.m.wikipedia.org/wiki/Valenzbindungstheorie

###### **Molecular Hamiltonian**

The Schrödinger equation can then be solved (usually approximately) for the electronic wavefunction for fixed nuclear distances. The plot of electronic energy as a function of nuclear distances is called the potential energy surface.
The P.E. surface represents the potential within which the nuclei move. Motion of nuclei on this surface corresponds to molecular vibration. (Rotation is considered as a special case of vibration in which internuclear distances are fixed.)

https://en.m.wikipedia.org/wiki/Molecular_Hamiltonian

###### **Born–Oppenheimer Approximation**

* [Born–Oppenheimer Approximation](https://en.m.wikipedia.org/wiki/Born–Oppenheimer_approximation) is the best known mathematical approximation in molecular dynamics

* it is the assumption that the **wave functions of atomic nuclei and electrons in a molecule can be treated separately**, based on the fact that the nuclei are much heavier than the electrons

* The approximation is **widely used in quantum chemistry to speed up the computation of molecular wavefunctions** and other properties for large molecules.

* There are **cases where the assumption of separable motion no longer holds**, which make the approximation lose validity (it is said to "break down"), but is then **often used as a starting point for more refined methods**.

* In molecular [spectroscopy](https://en.m.wikipedia.org/wiki/Spectroscopy), using the BO approximation means <font color="blue">**considering molecular energy as a sum of independent terms**</font>, e.g.: 

> $E_{\text{total}}=E_{\text{electronic}}+E_{\text{vibrational}}+E_{\text{rotational}}+E_{\text{nuclear spin}}$ 

* These terms are of different orders of magnitude and the nuclear spin energy is so small that it is often omitted. 

* The electronic energies $E_{\text{electronic}}$ consist of kinetic energies, interelectronic repulsions, internuclear repulsions, and electron–nuclear attractions, which are the terms typically included when computing the electronic structure of molecules.

* *Example: The benzene molecule consists of 12 nuclei and 42 electrons. The Schrödinger equation, which must be solved to obtain the energy levels and wavefunction of this molecule, is a partial differential eigenvalue equation in the three-dimensional coordinates of the nuclei and electrons, giving 3 × 12 + 3 × 42 = 36 nuclear + 126 electronic = 162 variables for the wave function. The computational complexity, i.e. the computational power required to solve an eigenvalue equation, increases faster than the square of the number of coordinates.*

**The Born-Oppenheimer Approximation states that electronic motion is so fast that
nuclei can be considered fixed**: the mass of a nuclear is so much greater than the mass of an electron, we set the nuclei as fixed, or the kinetic energy of the nuclei are zero (1 proton weighs about 1,800 times more than a single electron, and mist nuclei are even heavier than hydrogen nuclei, i.e. Carbon nucleus is 20,000 times heavier than an electron). We're going to solve of the wavefunction of the electrons at a fixed position of the nuclei.

The potential energy surface then corresponds to a plot of the energy of the system as a function of nuclear coordinates.


See also video: [Quantum Chemistry 10.1 - Hydrogen Molecule Hamiltonian](https://www.youtube.com/watch?v=BBoE6NRRZ8k)

See also Video: [Quantum Chemistry 10.2 - Born-Oppenheimer Approximation](https://www.youtube.com/watch?v=UGB2psWD69s)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_306.png)

###### **The Franck-Condon principle**

https://en.m.wikipedia.org/wiki/Franck–Condon_principle

Video: [The Franck-Condon principle](https://www.youtube.com/watch?v=mf_zFHxiY28)

###### **Potential energy surface**

The potential energy surface then corresponds to a plot of the energy of
the system as a function of nuclear coordinates.

https://en.m.wikipedia.org/wiki/Potential_energy_surface

###### **First & Second quantization & Quantum many-body states**

https://en.m.wikipedia.org/wiki/Second_quantization

###### **Fock States, Fock space & Vacuum State**

https://en.m.wikipedia.org/wiki/Fock_space

All the Fock states form a complete basis of the many-body Hilbert space, or Fock space.

The Fock state with all occupation numbers equal to zero is called the [vacuum state](https://en.m.wikipedia.org/wiki/Quantum_vacuum_state)

Video: [Fock space: variable number of quantum particles](https://www.youtube.com/watch?v=jAw9WMkcCj0)

* the first postulate of quantum mechanics tells us that all the action happens in a vector space called 'state space'

* in many situations all we need to consider is a fixed number of particles, ranging from one-dimensional problems to multi-particle systems where we had tensor product state spaces

* now we take this a step further and consider a state space that allows a variable number of particles = Fock space

* the Fock space is the state space in which quantum mechanics happens when we work in the second quantization formalism

* plays a big role in quantum field theory, quantum statistical mechanics and many more

* states with a well-defined number of particles are called Fock states

* a Fock state has a fixed number of particles and a Fock state belongs to a subspace of the Fock space $\mathcal{F}_N$

> $\mathcal{F}=\mathcal{F}_{0} \oplus \mathcal{F}_{1} \oplus \mathcal{F}_{2} \oplus \cdots \oplus \mathcal{F}_{N} \oplus \cdots$

* a very important Fock state is the 'vacuum state' with zero particles

* the power of Fock space is that it allows us to go beyond a fixed number of particles in Fock states: doing this by forming linear superpositions of states corresponding to different number of particles

* To navigate the Fock space: creation and annihilation operators

###### **Spectroscopy**

https://en.m.wikipedia.org/wiki/Spectroscopy

###### **More stuff**

[Quantum Chemistry 9.1 - Atomic Units](https://www.youtube.com/watch?v=93S-9Izr0nI)

https://slideplayer.com/slide/2770098/

MODULE 11

The extra electron adds the distances $r_{12}, r_{28}$, and $r_{2 A}$.

This adds complexity in that an electron-electron repulsion term appears, as well as two more electron-nucleon potential energy terms.

The hamiltonian for the hydrogen molecule in the Born-Oppenheimer approximation and in atomic units is

$
\hat{H}=-\frac{1}{2}\left(\nabla_{1}^{2}+\nabla_{2}^{2}\right)-\frac{1}{r_{1 A}}-\frac{1}{r_{18}}-\frac{1}{r_{2 A}}-\frac{1}{r_{2 B}}+\frac{1}{r_{12}}+\frac{1}{R}
$


https://www.youtube.com/watch?v=qc1TGwedSIc

[The Hy­dro­gen Mol­e­cule](https://web1.eng.famu.fsu.edu/~dommelen/quantum/style_a/hmol.html)

[Pennylane: Building molecular Hamiltonians](https://pennylane.ai/qml/demos/tutorial_quantum_chemistry.html)

[Pennylane: Optimization of molecular geometries](https://pennylane.ai/qml/demos/tutorial_mol_geo_opt.html)

[Pennylane: VQE in different spin sectors](https://pennylane.ai/qml/demos/tutorial_vqe_spin_sectors.html)

[Pennylane: Givens rotations for quantum chemistry](https://pennylane.ai/qml/demos/tutorial_givens_rotations.html)

[Chem Libretexts: Geometry_of_Molecules](https://chem.libretexts.org/Bookshelves/Physical_and_Theoretical_Chemistry_Textbook_Maps/Supplemental_Modules_(Physical_and_Theoretical_Chemistry)/Chemical_Bonding/Lewis_Theory_of_Bonding/Geometry_of_Molecules)


[Wiki: Molecular Geometry](https://en.m.wikipedia.org/wiki/Molecular_geometry)

[Wiki: (Hartree) atomic units](https://en.m.wikipedia.org/wiki/Hartree_atomic_units)

[Wiki: Born–Oppenheimer_approximation](https://en.m.wikipedia.org/wiki/Born–Oppenheimer_approximation)

[Wiki: Minimal Basis Sets (STO-NG-Basissätze)](https://de.m.wikipedia.org/wiki/STO-NG-Basissätze)

[Wiki: Molecular Orbital](https://en.m.wikipedia.org/wiki/Molecular_orbital)

[Wiki: Jordan-Wigner encoding (transformation)](https://de.m.wikipedia.org/wiki/Jordan-Wigner-Transformation)

[Wiki: Hartree-Fock (HF) state](https://en.wikipedia.org/wiki/Hartree–Fock_method)

### **<font color="blue">Computational Chemistry $\phi_i$**

###### **Density matrix renormalization group**

* The [density matrix renormalization group](https://en.m.wikipedia.org/wiki/Density_matrix_renormalization_group) (DMRG) is a numerical variational technique devised to obtain the low-energy physics of quantum many-body systems with high accuracy. 

* **As a variational method, DMRG is an efficient algorithm that attempts to find the lowest-energy matrix product state wavefunction of a Hamiltonian**. It was invented in 1992 by Steven R. White and it is nowadays the most efficient method for 1-dimensional systems.

###### **Fock Operator and Hartree–Fock Method**

*Task: Calculating Molecular Orbitals. Describe orbital energies & wavefunctions (of electrons in molecules) of many body systems (Vielteilchensysteme)*

Fock Operator Video: https://www.youtube.com/watch?v=3ph5XtkRVl0

Die Hartree-Fock-Gleichung lautet:

>$
\hat{F}\left|\phi_{m}\right\rangle=\varepsilon_{m}\left|\phi_{m}\right\rangle
$

mit dem Fock-Operator $\hat{F}:$

>$
\hat{F}_{i}=\hat{h}_{i}+\sum_{j=1}^{N_{\text {Elektr. }}}\left(2 \hat{J}_{j}-\hat{K}_{j}\right)
$

* Der Fock-Operator ist ein effektiver Ein-Elektronen-Operator, der die kinetische Energie eines Elektrons $i$, die Coulomb-Anziehung mit dem Kern $\left(\hat{h}_{i}\right)$ und die Coulomb-Abstoßung zu allen anderen Elektronen $j\left(\hat{J}_{j}\right.$ und $\hat{K}_{j}$ ) beschreibt. 

* $\hat{J}_{j}$ und $\hat{K}_{j}$ beschreiben als lokaler Coulomb-Operator und Austauschoperator die mittelwertartige Wechselwirkung mit allen anderen Teilchen. 

* Der Fock-Operator hängt somit von seinen eigenen Eigenfunktionen (da alle besetzten Orbitale bekannt sein müssen) ab, die Hartree-Fock-Gleichung ist somit nicht-linear, was eine iterative-Lösung notwendig macht. 

* Die Lösungen des SCF-Durchlaufs (Self-Consistent-Field-Methode) in HF sind die optimierten Orbitale $\phi _{m}$ als Eigenfunktionen des Fock-Operators mit den dazugehörigen Eigenenergien als Orbitalenergien $\varepsilon _{m}$.

In the [HF approximation](https://en.m.wikipedia.org/wiki/Hartree–Fock_method), each electron in the molecule is treated as an independent particle that moves under the influence of the Coulomb potential due to the nuclei, and a mean field generated by all other electrons. 

* The optimized coefficients are precisely what we need to build the second-quantized Hamiltonian.

* [Hartree-Fock-Methode](https://de.m.wikipedia.org/wiki/Hartree-Fock-Methode) ist eine probabilistische Parameterschätzung und ermöglicht es, Orbitalenergien und Wellenfunktionen von quantenmechanischen Vielteilchensystemen näherungsweise zu berechnen und ist eine so genannte Ab-initio-Methode, d. h. sie kommt ohne empirische Parameter aus und benötigt nur Naturkonstanten. Sie ist der Ausgangspunkt für Post-Hartree-Fock-Methoden, welche die Genauigkeit der Berechnungen verbessern.

Die Hartree-Fock-Methode ist die Basis der Molekülorbitaltheorie.

* Die Hartree-Fock-Methode ist die Basis der Molekülorbitaltheorie.

Die [Hartree-Fock-Methode](https://de.m.wikipedia.org/wiki/Hartree-Fock-Methode) geht von der zeitunabhängigen Schrödinger-Gleichung (hier in Dirac-Notation)

>$
\hat{H}|\psi\rangle=E|\psi\rangle
$

aus, welche die Energie $E$ eines Systems aus der Wellenfunktion $|\psi\rangle$ berechnet, indem die Eigenwerte des Hamilton-Operators $\hat{H}$ zu dieser Wellenfunktion gesucht werden. 

* Im Hamilton-Operator werden alle Energiebeiträge der Teilchen und Felder im System sowie deren Wechselwirkungen untereinander beschrieben. 

* In vielen praktisch wichtigen Systemen (wie z. B. den Elektronen in einem Molekül) sind die Teilchen miteinander korreliert und beeinflussen sich gegenseitig. Dadurch kann die Schrödinger-Gleichung für solche Systeme nicht mehr exakt, sondern nur noch näherungsweise gelöst werden.

In computational physics and chemistry, the [Hartree–Fock (HF) method](https://en.m.wikipedia.org/wiki/Hartree%E2%80%93Fock_method) is a method of approximation for the determination of the wave function and the energy of a [quantum many-body system](https://en.m.wikipedia.org/wiki/Many-body_problem) <u>**in a stationary state**</u>.

* The Hartree–Fock method often assumes that the exact N-body wave function of the system can be approximated 

  * by a single [Slater determinant](https://en.m.wikipedia.org/wiki/Slater_determinant) (in the case where the particles are fermions) or 
  
  * by a single [permanent](https://en.m.wikipedia.org/wiki/Permanent_(mathematics)) (in the case of bosons) of N [spin-orbitals](https://en.m.wikipedia.org/wiki/Atomic_orbital). 

* By invoking the [variational method](https://en.m.wikipedia.org/wiki/Calculus_of_variations), one can derive a set of N-coupled equations for the N spin orbitals. A solution of these equations yields the Hartree–Fock wave function and energy of the system.


Die Hartree-Fock-Methode vereinfacht die Wechselwirkungen der Teilchen untereinander so, dass diese nicht mehr jeweils paarweise untereinander wechselwirken, sondern mit einem [Feld](https://de.m.wikipedia.org/wiki/Feld_(Physik)), das von allen anderen Teilchen im Mittelwert erzeugt wird – dem so genannten mean field (mittleren Feld). Das Feld hängt zwar immer noch vom Verhalten der einzelnen Teilchen ab, die Lösung kann aber jetzt schrittweise berechnet werden:

1. Ein Ausgangszustand wird ausgewählt und daraus das Feld erzeugt.

2. Mit diesem wird dann die Schrödingergleichung für jedes einzelne Teilchen gelöst.

3. Zusammengenommen ergeben die einzelnen Lösungen dann einen neuen Zustand und ein neues Feld.

Dieser Vorgang wird wiederholt, bis sich aufeinanderfolgende Lösungen nur mehr geringfügig unterscheiden, das Feld also zu Lösungen führt, die das Feld selbst konsistent wieder erzeugen. Daraus leitet sich der Begriff self-consistent field ab, der für diesen Teil der Hartree-Fock-Methode verwendet wird.

Als Wellenfunktionen für die behandelten Vielteilchensysteme werden bei Bosonen ein symmetrisches (Hartree-)Produkt von Einteilchenwellenfunktionen verwendet, bei Fermionen (wie Elektronen, Protonen und Neutronen) eine antisymmetrische Kombination dieser Produkte (eine sogenannte Slater-Determinante).

* Um die Schrödingergleichung zu lösen, werden diese Einteilchenwellenfunktionen so variiert, dass die aus der Gleichung entstehende Energie minimal wird. Aufgrund des [Rayleigh-Ritz-Prinzips](https://de.m.wikipedia.org/wiki/Rayleigh-Ritz-Prinzip) (=Variationsprinzip zur Bestimmung des kleinsten Eigenwerts eines Eigenwertproblems) ist diese Energie dann eine obere Grenze für die tatsächliche Energie des Systems. 

* Die dadurch berechnete Wellenfunktion des gesamten Systems ist allerdings nicht notwendigerweise eine Annäherung der tatsächlichen Wellenfunktion.


###### **Ritz Method**

* The [Ritz method](https://en.wikipedia.org/wiki/Ritz_method) is a direct method to find an approximate solution for [boundary value problems](https://en.wikipedia.org/wiki/Boundary_value_problem). The method is named after Walther Ritz, although also commonly called the [Rayleigh–Ritz method](https://en.wikipedia.org/wiki/Rayleigh–Ritz_method) and the [Ritz-Galerkin method](https://en.wikipedia.org/wiki/Galerkin_method).

* In quantum mechanics, a system of particles can be described in terms of an "energy functional" or Hamiltonian, which will measure the energy of any proposed configuration of said particles. 
  
  * It turns out that certain privileged configurations are more likely than other configurations, and this has to do with the eigenanalysis ("analysis of characteristics") of this Hamiltonian system.
  
  * Because it is often impossible to analyze all of the infinite configurations of particles to find the one with the least amount of energy, it becomes essential to be able to approximate this Hamiltonian in some way for the purpose of numerical computations.

* The Ritz method can be used to achieve this goal. In the language of mathematics, it is exactly the [finite element method](https://en.wikipedia.org/wiki/Finite_element_method) used to compute the eigenvectors and eigenvalues of a Hamiltonian system.


https://www.youtube.com/watch?v=w4ijyrdLbNA

https://en.m.wikipedia.org/wiki/Ritz_method

https://en.m.wikipedia.org/wiki/Rayleigh%E2%80%93Ritz_method

###### **Self-consistent field (SCF) method**

Video: [Computational Chemistry 4.27 - Self-Consistent Field](https://www.youtube.com/watch?v=x-O3_t4GVy8)

Die [Self-Consistent-Field-Methode (SCF)](https://de.m.wikipedia.org/wiki/Self-Consistent-Field-Methode) ist ein iteratives Lösungsverfahren in der Computerchemie. SCF ist eine von vielen Möglichkeiten zur Lösung von Molekülberechnungen. Das self-consistent-field (dt. selbstkonsistentes Feld) ist eine Menge von Funktionen, die eine Eigenwertgleichung löst.

Da die Fock-Matrix von ihren eigenen Lösungen abhängt muss sie iterativ gelöst werden, dies geschieht mit der SCF-Methode. Ein typischer SCF-Ablauf sieht wie folgt aus:

1. Berechnung aller Ein- und Zweielektronenintegrale.
2. Erzeugung geeigneter MO-Koeffizienten.
3. Bildung der anfänglichen Dichte-Matrix.
4. Bildung der Fock-Matrix als Ein-Elektron-Integrale und der Dichtematrix mal der Zwei-Elektronen-Integrale.
5. Diagonalisierung der Fock-Matrix. Als Ergebnis enthalten die Eigenvektoren die neuen MO-Koeffizienten.
6. Bildung einer neuen Dichte-Matrix. Wenn sich die neue Dichte-Matrix und die anfängliche Dichte-Matrix gleichen kann der SCF-Durchlauf beendet werden, wenn nicht beginnt der Zyklus wieder bei Schritt 4 mit der neuen Dichte-Matrix.

###### **Time-evolving block decimation**

https://en.m.wikipedia.org/wiki/Time-evolving_block_decimation

###### **Post–Hartree–Fock**

Video: [Quantum Chemistry 9.11 - Post Hartree-Fock Methods](https://www.youtube.com/watch?v=oU1zO2bPPB4)

In computational chemistry, [post–Hartree–Fock methods](https://en.m.wikipedia.org/wiki/Post%E2%80%93Hartree%E2%80%93Fock) are the set of **methods developed to improve on the Hartree–Fock (HF), or self-consistent field (SCF) method**. 

**They add electron correlation which is a more accurate way of including the repulsions between electrons** than in the Hartree–Fock method where repulsions are only averaged.



**In general, the SCF procedure makes several assumptions** about the nature of the multi-body Schrödinger equation and its set of solutions:

1. For molecules, the Born–Oppenheimer approximation is inherently assumed. The true wavefunction should also be a function of the coordinates of each of the nuclei.

2. Typically, relativistic effects are completely neglected. The momentum operator is assumed to be completely nonrelativistic.

3. The basis set is composed of a finite number of orthogonal functions. The true wavefunction is a linear combination of functions from a complete (infinite) basis set.

4. The energy eigenfunctions are assumed to be products of one-electron wavefunctions. The effects of electron correlation, beyond that of exchange energy resulting from the anti-symmetrization of the wavefunction, are completely neglected.

For the great majority of systems under study, in particular for excited states and processes such as molecular dissociation reactions, **the fourth item is by far the most important**. As a result, the term post–Hartree–Fock method is typically used for methods of approximating the electron correlation of a system.

Usually, post–Hartree–Fock methods give more accurate results than Hartree–Fock calculations, although the added accuracy comes with the price of added computational cost.

* Configuration interaction (CI)

* Coupled cluster (CC)

* Multi-configuration time-dependent Hartree (MCTDH)

* Møller–Plesset perturbation theory (MP2, MP3, MP4 etc.)

* Quadratic configuration interaction (QCI)

* Quantum chemistry composite methods (G2, G3, CBS, T1. etc.)

Related Methods that use more than one determinant are not strictly post–Hartree–Fock methods, as they use a single determinant as reference, but they often use similar perturbation, or configuration interaction methods to improve the description of electron correlation. These methods include:

* Multi-configurational self-consistent field (MCSCF)

* Multireference single and double configuration interaction (MRCISD)

* N-electron valence state perturbation theory (NEVPT).

###### **Multi-configuration time-dependent Hartree (MCTDH)**

https://en.m.wikipedia.org/wiki/Multi-configuration_time-dependent_Hartree

###### **Configuration interaction**

[Configuration interaction (CI)](https://en.m.wikipedia.org/wiki/Configuration_interaction) is a post-Hartree–Fock linear variational method for solving the nonrelativistic Schrödinger equation within the Born–Oppenheimer approximation for a quantum chemical multi-electron system. Mathematically, **configuration simply describes the linear combination of Slater determinants used for the wave function.**

In terms of a specification of orbital occupation (for instance,
$\left.(1 \mathrm{~s})^{2}(2 \mathrm{~s})^{2}(2 \mathrm{p})^{1} \ldots\right)$, interaction means the mixing (interaction) of
different electronic configurations (states). 

* Due to the long
$\mathrm{CPU}$ time and large memory required for $\mathrm{Cl}$ calculations, the
method is limited to relatively small systems.

* In contrast to the Hartree-Fock method, in order to account for
electron correlation, $\mathrm{Cl}$ **uses a variational wave function that is
a linear combination of configuration state functions (CSFs)
built from spin orbitals (denoted by the superscript SO)**,

> $\Psi=\sum_{I=0} c_{I} \Phi_{I}^{S O}=c_{0} \Phi_{0}^{S O}+c_{1} \Phi_{1}^{S O}+\ldots$

* where $\Psi$ is usually the electronic ground state of the system. 

* If
the expansion includes all possible CSFs of the appropriate
symmetry, then this is a full configuration interaction

###### **Density Functional theory / Dichtefunktionaltheorie (DFT)**

https://de.m.wikipedia.org/wiki/Dichtefunktionaltheorie_(Quantenphysik)

https://www.chemistryworld.com/news/algorithm-out-of-googles-deepmind-finesses-dft-calculations/4014939.article


###### **LCAO-Methode**

https://de.m.wikipedia.org/wiki/LCAO-Methode

###### **Molekularfeldtheorie (Mean Field Theory)**

https://de.m.wikipedia.org/wiki/Molekularfeldtheorie